<a href="https://colab.research.google.com/github/sitivio/vio/blob/main/PreprocessingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import xgboost as xgb

In [ ]:
data = pd.read_csv('labeling.csv', sep=',', encoding='utf-8')
data.head()

,value,comment
0,negatif,"Setelah update, justru ada masalah baru. Waktu..."
1,negatif,Tolong di perbaiki bug nya. Kesel bngt iklan m...
2,positif,Super... Kita dapat menemukan konten sesuai mi...
3,negatif,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...
4,negatif,"Ini mulai ada iklan, menurut aku sih gapapa ka..."


In [ ]:
#presos case folding
import re
def casefolding(comment):
  comment = comment.lower()
  comment = comment.strip(" ")
  comment = re.sub(r'[?|$|.|!2_:")(-+,]', '', comment)
  return comment
data['comment_casefolding'] = data['comment'].apply(casefolding)
data.head()

,value,comment,comment_casefolding
0,negatif,"Setelah update, justru ada masalah baru. Waktu...",setelah update justru ada masalah baru waktu m...
1,negatif,Tolong di perbaiki bug nya. Kesel bngt iklan m...,tolong di perbaiki bug nya kesel bngt iklan mu...
2,positif,Super... Kita dapat menemukan konten sesuai mi...,super kita dapat menemukan konten sesuai minat...
3,negatif,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...,kecewa ahh sekarang iklan tik tok kok ga berhe...
4,negatif,"Ini mulai ada iklan, menurut aku sih gapapa ka...",ini mulai ada iklan menurut aku sih gapapa kar...


In [ ]:
import re

def cleancomment(comment):
  comment = re.sub('@[A-Za-z0–9]+', '', comment) #Removing @mentions
  comment = re.sub('#', '', comment) # Removing '#' hash tag
  comment = re.sub('RT[\s]+', '', comment) # Removing RT
  comment = re.sub('https?:\/\/\S+', '', comment) # Removing hyperlink
  comment = re.sub(r'\d+', '', comment)
  return comment

data['comment'] = data['comment'].apply(cleancomment)

def remove_numbers(comment):
  # Menggunakan ekspresi reguler untuk mencocokkan angka dan menghapusnya
  return re.sub(r'\d+', '', comment)

def remove_emojis(comment):
 emoji_pattern = re.compile("["
     u"\U0001F600-\U0001F64F" # emoticons
     u"\U0001F300-\U0001F5FF" # symbols & pictographs
     u"\U0001F680-\U0001F6FF" # transport & map symbols
     u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                     "]+", flags=re.UNICODE)
 return re.sub(emoji_pattern, '', comment)

data['comment_cleans'] = data['comment'].apply(remove_emojis)
data.head()

,value,comment,comment_casefolding,comment_cleans
0,negatif,"Setelah update, justru ada masalah baru. Waktu...",setelah update justru ada masalah baru waktu m...,"Setelah update, justru ada masalah baru. Waktu..."
1,negatif,Tolong di perbaiki bug nya. Kesel bngt iklan m...,tolong di perbaiki bug nya kesel bngt iklan mu...,Tolong di perbaiki bug nya. Kesel bngt iklan m...
2,positif,Super... Kita dapat menemukan konten sesuai mi...,super kita dapat menemukan konten sesuai minat...,Super... Kita dapat menemukan konten sesuai mi...
3,negatif,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...,kecewa ahh sekarang iklan tik tok kok ga berhe...,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...
4,negatif,"Ini mulai ada iklan, menurut aku sih gapapa ka...",ini mulai ada iklan menurut aku sih gapapa kar...,"Ini mulai ada iklan, menurut aku sih gapapa ka..."


In [ ]:
#proses tokenizing
def token(comment):
  nstr = comment.split(' ')
  dat= []
  a = -1
  for hu in nstr:
    a = a + 1
  if hu == '':
    dat.append(a)
  p = 0
  b = 0
  for q in dat:
    b = q - p
    del nstr[b]
    p = p + 1
  return nstr
data['comment_tokens'] = data['comment_casefolding'].apply(token)
data.head()

,value,comment,comment_casefolding,comment_cleans,comment_tokens
0,negatif,"Setelah update, justru ada masalah baru. Waktu...",setelah update justru ada masalah baru waktu m...,"Setelah update, justru ada masalah baru. Waktu...","[setelah, update, justru, ada, masalah, baru, ..."
1,negatif,Tolong di perbaiki bug nya. Kesel bngt iklan m...,tolong di perbaiki bug nya kesel bngt iklan mu...,Tolong di perbaiki bug nya. Kesel bngt iklan m...,"[tolong, di, perbaiki, bug, nya, kesel, bngt, ..."
2,positif,Super... Kita dapat menemukan konten sesuai mi...,super kita dapat menemukan konten sesuai minat...,Super... Kita dapat menemukan konten sesuai mi...,"[super, kita, dapat, menemukan, konten, sesuai..."
3,negatif,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...,kecewa ahh sekarang iklan tik tok kok ga berhe...,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...,"[kecewa, ahh, sekarang, iklan, tik, tok, kok, ..."
4,negatif,"Ini mulai ada iklan, menurut aku sih gapapa ka...",ini mulai ada iklan menurut aku sih gapapa kar...,"Ini mulai ada iklan, menurut aku sih gapapa ka...","[ini, mulai, ada, iklan, menurut, aku, sih, ga..."


In [ ]:
!pip install stop-words

In [ ]:
#proses filtering
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def stopword_removal(comment):
  filtering = stopwords.words('indonesian','english')
  x = []
  data = []
  def myFunc(x):
    if x in filtering:
      return False
    else:
      return True
  fit = filter(myFunc, comment)
  for x in fit:
    data.append(x)
  return data
data['comment_filtering'] = data['comment_tokens'].apply(stopword_removal)
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,value,comment,comment_casefolding,comment_cleans,comment_tokens,comment_filtering
0,negatif,"Setelah update, justru ada masalah baru. Waktu...",setelah update justru ada masalah baru waktu m...,"Setelah update, justru ada masalah baru. Waktu...","[setelah, update, justru, ada, masalah, baru, ...","[update, menonton, video, iklan, muncul, setal..."
1,negatif,Tolong di perbaiki bug nya. Kesel bngt iklan m...,tolong di perbaiki bug nya kesel bngt iklan mu...,Tolong di perbaiki bug nya. Kesel bngt iklan m...,"[tolong, di, perbaiki, bug, nya, kesel, bngt, ...","[tolong, perbaiki, bug, nya, kesel, bngt, ikla..."
2,positif,Super... Kita dapat menemukan konten sesuai mi...,super kita dapat menemukan konten sesuai minat...,Super... Kita dapat menemukan konten sesuai mi...,"[super, kita, dapat, menemukan, konten, sesuai...","[super, menemukan, konten, sesuai, minat, baka..."
3,negatif,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...,kecewa ahh sekarang iklan tik tok kok ga berhe...,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...,"[kecewa, ahh, sekarang, iklan, tik, tok, kok, ...","[kecewa, ahh, iklan, tik, tok, ga, berhenti, k..."
4,negatif,"Ini mulai ada iklan, menurut aku sih gapapa ka...",ini mulai ada iklan menurut aku sih gapapa kar...,"Ini mulai ada iklan, menurut aku sih gapapa ka...","[ini, mulai, ada, iklan, menurut, aku, sih, ga...","[iklan, sih, gapapa, , detik, doang, skip, tao..."


In [ ]:
!pip install swifter
import swifter
!pip install PySastrawi

In [ ]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data['comment_filtering']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])

print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data['comment_stemmed'] = data['comment_filtering'].swifter.apply(get_stemmed_term)
print(data['comment_stemmed'])

3794
------------------------
update : update
menonton : tonton
video : video
iklan : iklan
muncul : muncul
setalah : seta
skip : skip
iklannya : iklan
hilang : hilang
beda : beda
yg : yg
suara : suara
nyangkut : nyangkut
scroll : scroll
mengganggu : ganggu
kenyamanan : nyaman
tolong : tolong
perbaiki : baik
bug : bug
nya : nya
kesel : kesel
bngt : bngt
udh : udh
tpi : tpi
muter : muter
latar : latar
loop : loop
ngulang : ngulang
swipe : swipe
jdi : jdi
double : double
 : 
tutup : tutup
aplikasi : aplikasi
ilang : ilang
sampe : sampe
skrng : skrng
gk : gk
super : super
menemukan : temu
konten : konten
sesuai : sesuai
minat : minat
bakat : bakat
edit : edit
foto : foto
memudahkan : mudah
jual : jual
beli : beli
barang : barang
belanja : belanja
produk : produk
ulasan : ulas
langsung : langsung
kreator : kreator
unggah : unggah
mudah : mudah
tampilan : tampil
halus : halus
detail : detail
ditampilkan : tampil
kualitas : kualitas
sempurna : sempurna
kecewa : kecewa
ahh : ahh
tik : tik
tok

Pandas Apply:   0%|          | 0/1200 [00:00<?, ?it/s]

0       [update, tonton, video, iklan, muncul, seta, s...
1       [tolong, baik, bug, nya, kesel, bngt, iklan, m...
2       [super, temu, konten, sesuai, minat, bakat, ap...
3       [kecewa, ahh, iklan, tik, tok, ga, henti, kecu...
4       [iklan, sih, gapapa, , detik, doang, skip, tao...
                              ...                        
1195    [update, maleh, goblokmendopekok, benakno, teh...
1196    [kesini, ngelag, hpku, sampe, mati, mati, asik...
1197                 [ken, akun, tik, tok, ku, rame, min]
1198    [tiktok, babi, komen, anggap, langgar, china, ...
1199                            [lupa, folow, lal, stroy]
Name: comment_stemmed, Length: 1200, dtype: object


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Fungsi stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)

data['comment_tokens'] = data['comment'].apply(stemmed_wrapper)

# Stemming pada setiap kata
data['comment_stemmed'] = data['comment_tokens'].apply(stemmed_wrapper)

print(data[['comment', 'comment_tokens', 'comment_stemmed']])


                                                comment  \
0     Setelah update, justru ada masalah baru. Waktu...   
1     Tolong di perbaiki bug nya. Kesel bngt iklan m...   
2     Super... Kita dapat menemukan konten sesuai mi...   
3     Kecewa ahh.. Sekarang iklan tik tok kok ga ber...   
4     Ini mulai ada iklan, menurut aku sih gapapa ka...   
...                                                 ...   
1195  Setelah update maleh goblok,Mendo,pekok benakn...   
1196  Makin kesini makin ngelag. Hpku sampe mati mat...   
1197                    Pengen Akun tik tok ku rame min   
1198  Tiktok babi memang kalian, saya komen hal yang...   
1199                        Jangan lupa folow LAL Stroy   

                                         comment_tokens  \
0     telah update justru ada masalah baru waktu ton...   
1     tolong di baik bug nya kesel bngt iklan muncul...   
2     super kita dapat temu konten sesuai minat dan ...   
3     kecewa ahh sekarang iklan tik tok kok ga henti...

In [ ]:
data.head()

,value,comment,comment_casefolding,comment_cleans,comment_tokens,comment_filtering,comment_stemmed
0,negatif,"Setelah update, justru ada masalah baru. Waktu...",setelah update justru ada masalah baru waktu m...,"Setelah update, justru ada masalah baru. Waktu...",telah update justru ada masalah baru waktu ton...,"[update, menonton, video, iklan, muncul, setal...","[update, tonton, video, iklan, muncul, seta, s..."
1,negatif,Tolong di perbaiki bug nya. Kesel bngt iklan m...,tolong di perbaiki bug nya kesel bngt iklan mu...,Tolong di perbaiki bug nya. Kesel bngt iklan m...,tolong di baik bug nya kesel bngt iklan muncul...,"[tolong, perbaiki, bug, nya, kesel, bngt, ikla...","[tolong, baik, bug, nya, kesel, bngt, iklan, m..."
2,positif,Super... Kita dapat menemukan konten sesuai mi...,super kita dapat menemukan konten sesuai minat...,Super... Kita dapat menemukan konten sesuai mi...,super kita dapat temu konten sesuai minat dan ...,"[super, menemukan, konten, sesuai, minat, baka...","[super, temu, konten, sesuai, minat, bakat, ap..."
3,negatif,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...,kecewa ahh sekarang iklan tik tok kok ga berhe...,Kecewa ahh.. Sekarang iklan tik tok kok ga ber...,kecewa ahh sekarang iklan tik tok kok ga henti...,"[kecewa, ahh, iklan, tik, tok, ga, berhenti, k...","[kecewa, ahh, iklan, tik, tok, ga, henti, kecu..."
4,negatif,"Ini mulai ada iklan, menurut aku sih gapapa ka...",ini mulai ada iklan menurut aku sih gapapa kar...,"Ini mulai ada iklan, menurut aku sih gapapa ka...",ini mulai ada iklan turut aku sih gapapa karen...,"[iklan, sih, gapapa, , detik, doang, skip, tao...","[iklan, sih, gapapa, , detik, doang, skip, tao..."


In [ ]:
data.to_csv('data_preprocessing.csv')